# ML Simulation Training
In this notebook you are going to train a ML policy.

However, you won't use examples from the SDV as data, but other agents around it instead.

This may sound like a small difference, but it has profound implications still:
- by using data from multiple sources you're **including much more variability in your training data**;
- two agents may have taken different choices at the same intersection, leading to **multi-modal data**;
- the **quality of the annotated data is expected to be sensibility lower** compared to the SDV, as we're leveraging a perception system.

Still, the final prize is even better than planning as this policy can potentially drive all the agents in the scene and it's not limited to the SDV only.

![simulation-example](https://github.com/woven-planet/l5kit/blob/master/docs/images/simulation/simulation_example.svg?raw=1)


In [ ]:
%%writefile setup_notebook_colab.sh
#!/bin/bash

# Make a temporary download folder
TEMP_DOWNLOAD_DIR=$(mktemp -d)
TEMP_DATASET_DIR=$(mktemp -d)

# Download sample zarr
echo "Downloading sample zarr dataset..."
wget https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/prediction/v1.1/sample.tar \
    -q --show-progress -P $TEMP_DOWNLOAD_DIR

mkdir -p $TEMP_DATASET_DIR/scenes
tar xf $TEMP_DOWNLOAD_DIR/sample.tar -C $TEMP_DATASET_DIR/scenes

# Download semantic map
echo "Downloading semantic map..."
wget https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/prediction/v1.1/semantic_map.tar \
    -q --show-progress -P $TEMP_DOWNLOAD_DIR
mkdir -p $TEMP_DATASET_DIR/semantic_map
tar xf $TEMP_DOWNLOAD_DIR/semantic_map.tar -C $TEMP_DATASET_DIR/semantic_map
cp $TEMP_DATASET_DIR/semantic_map/meta.json $TEMP_DATASET_DIR/meta.json

wget https://raw.githubusercontent.com/woven-planet/l5kit/master/examples/simulation/config.yaml -q

# Install L5Kit
echo "Installing L5kit..."
pip install --progress-bar off --quiet -U l5kit pyyaml
pip install ray==2.0.0rc1 --quiet
pip install "ray[air]" --quiet
pip install -U wandb --quiet

echo "Dataset and L5kit are ready !"
echo $TEMP_DATASET_DIR > "dataset_dir.txt"

In [ ]:
#@title Download L5 Sample Dataset and install L5Kit
#TODO: Place in own step and log to wandb as data logging step
import os
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !sh ./setup_notebook_colab.sh
    os.environ["L5KIT_DATA_FOLDER"] = open("./dataset_dir.txt", "r").read().strip()
else:
    print("Not running in Google Colab.")
    os.environ["L5KIT_DATA_FOLDER"] = "/tmp/l5kit_data"

sample.tar          100%[===================>]  51.37M  24.6MB/s    in 2.1s    
semantic_map.tar    100%[===================>]   2.86M  4.52MB/s    in 0.6s    
Installing L5kit...









ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
cmdstanpy 1.0.4 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
Dataset and L5kit are ready !
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 1.2 MB/s 
     |████████████████████████████████| 4.1 MB 49.2 MB/s 
     |████████████████████████████████| 8.8 MB 56.0 MB/s 
     |████████████████████████████████| 46

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 181 kB 63.1 MB/s 
     |████████████████████████████████| 157 kB 73.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 157 kB 58.5 MB/s 
     |████████████████████████████████| 157 kB 58.8 MB/s 
     |████████████████████████████████| 157 kB 55.9 MB/s 
     |████████████████████████████████| 157 kB 57.5 MB/s 
     |████████████████████████████████| 156 kB 11.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=b7b090365b6e212cdba0005f54ae368d3a97de5767b4b77096f780757ef437b6
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.geometry import transform_points
from l5kit.visualization import TARGET_POINTS_COLOR, draw_trajectory
from l5kit.planning.rasterized.model import RasterizedPlanningModel

import os

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
# get config
cfg = load_config_data("./config.yaml")
sample = True
if sample:
    cfg["train_data_loader"]["key"] = "scenes/sample.zarr"

In [ ]:
# # plot some examples
# for idx in range(0, len(train_dataset), len(train_dataset) // 10):
#     data = train_dataset[idx]
#     im = rasterizer.to_rgb(data["image"].transpose(1, 2, 0))
#     target_positions = transform_points(data["target_positions"], data["raster_from_agent"])
#     draw_trajectory(im, target_positions, TARGET_POINTS_COLOR)
#     plt.imshow(im)
#     plt.axis('off')
#     plt.show()

# Prepare for training
Our `AgentDataset` inherits from PyTorch `Dataset`; so we can use it inside a `Dataloader` to enable multi-processing.

In [ ]:
import ray.train as train
from ray.air import session

In [ ]:
def train_simulation_model(cfg: dict):

    dm = LocalDataManager(None)

    # rasterisation
    rasterizer = build_rasterizer(cfg, dm)

    # ===== INIT DATASET
    train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()
    train_dataset = AgentDataset(cfg, train_zarr, rasterizer)


    model_arch = cfg["model_params"]["model_architecture"]
    future_num_frames = cfg["model_params"]["future_num_frames"]

    train_cfg = cfg["train_data_loader"]
    shuffle = train_cfg["shuffle"]
    batch_size = train_cfg["batch_size"]
    num_workers = train_cfg["num_workers"]

    max_num_steps = cfg["train_params"]["max_num_steps"]

    batch_size_per_worker = batch_size // session.get_world_size()

    train_dataloader = DataLoader(train_dataset, shuffle=shuffle, batch_size=batch_size_per_worker, num_workers=num_workers)
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)

    model = RasterizedPlanningModel(
        model_arch=model_arch,
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * future_num_frames,  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
        criterion=nn.MSELoss(reduction="none")
    )
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # model = model.to(device)

    model = train.torch.prepare_model(model)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    tr_it = iter(train_dataloader)
    progress_bar = tqdm(range(max_num_steps))
    losses_train = []
    model.train()
    torch.set_grad_enabled(True)

    for _ in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
        data = {k: v for k, v in data.items()} #Quick hack to access data TODO: Replace with proper data loading with DDP
        loss = train_simulation_model_epoch(data, model, optimizer)
        losses_train.append(loss.item())
        progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

In [ ]:
def train_simulation_model_epoch(data, model, optimizer):
     # Forward pass        
    result = model(data)
    loss = result["loss"]
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
from ray.train.torch import TorchTrainer
from ray.air.config import RunConfig, ScalingConfig
from ray.air.callbacks.wandb import WandbLoggerCallback

In [ ]:
wandb_project = "level-5-sim"

In [ ]:
trainer = TorchTrainer(
    train_loop_per_worker=train_simulation_model,
    train_loop_config=cfg,
    scaling_config=ScalingConfig(num_workers=1, use_gpu=True), #TODO: Add logic to check if GPU is available here
    run_config=RunConfig(
            callbacks=[
                # This is the part needed to enable logging to Weights & Biases.
                # It assumes you've logged in before, e.g. with `wandb login`.
                WandbLoggerCallback(
                    project=wandb_project,
                    save_checkpoints=False,
                )
            ]
        )
)

In [ ]:
result = trainer.fit()
print(f"Last result: {result.metrics}")

2022-08-16 21:08:56,903	INFO wandb.py:119 -- Already logged into W&B.


Trial name,status,loc
TorchTrainer_a424f_00000,RUNNING,172.28.0.2:931


(RayTrainWorker pid=1019) 2022-08-16 21:12:02,949	INFO config.py:72 -- Setting up process group for: env:// [rank=0, world_size=1]


# Training loop
Here, we purposely include a barebone training loop. Clearly, many more components can be added to enrich logging and improve performance, such as:
- learning rate drop;
- loss weights tuning;
- importance sampling

To name a few.


Still, the sheer size of our dataset ensures that a reasonable performance can be obtained even with this simple loop.

In [ ]:
result

Result(metrics={'trial_id': 'a424f_00000', 'done': True}, error=None, log_dir=PosixPath('/root/ray_results/TorchTrainer_2022-08-16_21-08-56/TorchTrainer_a424f_00000_0_2022-08-16_21-08-59'))

### Plot the train loss curve
We can plot the train loss against the iterations (batch-wise) to check if our model has converged.

In [ ]:
# plt.plot(np.arange(len(losses_train)), losses_train, label="train loss")
# plt.legend()
# plt.show()

# Store the model

Let's store the model as a torchscript. This format allows us to re-load the model and weights without requiring the class definition later.

**Take note of the path, you will use it later to evaluate your planning model!**

In [ ]:
# to_save = torch.jit.script(model.cpu())
# path_to_save = f"{gettempdir()}/simulation_model.pt"
# to_save.save(path_to_save)
# print(f"MODEL STORED at {path_to_save}")

# Congratulations in training your first ML policy for simulation!
### What's Next

Now that your model is trained and safely stored, you can use it to control the agents around ego. We have a notebook just for that.

### [Simulation evaluation](./simulation_test.ipynb)
In this notebook a `planning_model` will control the SDV, while the `simulation_model` you just trained will be used for all other agents.

Don't worry if you don't have the resources required to train a model, we provide pre-trained models just below.

## Pre-trained models
we provide a collection of pre-trained models for the simulation task:
- [simulation model](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/simulation_models/simulation_model_20210416_5steps.pt) trained on agents over the semantic rasteriser with history of 0.5s;
- [planning model](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/simulation_models/planning_model_20210421_5steps.pt) trained on the AV over the semantic rasteriser with history of 0.5s;

To use one of the models simply download the corresponding `.pt` file and load it in the evaluation notebooks.